**Topic Analysis using LDA**



In [29]:
# Import Libraries 
import pandas as pd
import snscrape.modules.twitter as sntwitter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.utils import simple_preprocess
import gensim.downloader as api
import gensim.corpora as corpora

import nltk
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import string
import re
import textblob
from textblob import TextBlob

from wordcloud import WordCloud, STOPWORDS
from wordcloud import ImageColorGenerator
from PIL import Image
from emot.emo_unicode import UNICODE_EMOJI

porter = PorterStemmer()
lemmatizer = WordNetLemmatizer() 

import pyLDAvis.gensim_models 
import pickle 
import pyLDAvis
import os

import warnings
from pprint import pprint
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pdhiman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pdhiman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pdhiman\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\pdhiman\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\pdhiman\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


**Using pickle to get the NLP Preproccesed tweets**

In [30]:
# Loading All the tweets 

df = pd.read_pickle('source/nlp_sentiments.pkl')
df.head()

,Unnamed: 0,Date,ID,location,tweet,num_of_likes,num_of_retweet,language,cleaned_tweets,final_cleaned_tweets,sentiment_Original,sentiment_cleaned,sentiment_nlpfinal,vader_polarity
0,0,2022-11-07 23:59:59+00:00,1589769667765469186,"California, USA",Taking into account personal contributions &am...,2,1,en,taking account personal bad everyone better tr...,taking account personal bad everyone better tr...,positive,positive,positive,"{'neg': 0.271, 'neu': 0.403, 'pos': 0.326, 'co..."
1,1,2022-11-07 23:59:59+00:00,1589769667652235267,@jlo follows ♡ 01.29.21,whats your fav song?\n\n❥ I’m voting #Jennifer...,0,10,en,whats song voting,whats song voting,positive,neutral,neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,2,2022-11-07 23:59:59+00:00,1589769667127934977,Unknown,@MayoIsSpicyy He is allowed to speak his opini...,0,0,en,speak opinion like rest us opinion vote republ...,speak opinion like rest u opinion vote republi...,negative,positive,positive,"{'neg': 0.0, 'neu': 0.828, 'pos': 0.172, 'comp..."
3,3,2022-11-07 23:59:59+00:00,1589769666918244352,USA,HEY NY DISTRICT 10! PLEASE VOTE FOR @danielsgo...,1,1,en,hey district please vote,hey district please vote,positive,positive,positive,"{'neg': 0.0, 'neu': 0.566, 'pos': 0.434, 'comp..."
4,4,2022-11-07 23:59:59+00:00,1589769666679144448,DMV,@YDanasmithdutra @BaddCompani @politicalblond ...,3,0,en,vote blue matter,vote blue matter,negative,neutral,neutral,"{'neg': 0.0, 'neu': 0.645, 'pos': 0.355, 'comp..."


Data Preproccessing 
- Clean Null Values
- Stopwords
- Punctuation Removal
- Removing Emojis
- Removing URLs and Usernames
- Removing Hashtags
- Lemmatization
- Remove Special characters (amp)

Preparing for LDA Analysis


In [31]:
# Converting document (in our case tweets)  to array for doc2bow to process and create bag of word Corpus. 

def sent2words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data = df.final_cleaned_tweets.values.tolist()
data_words = list(sent2words(data))

In [32]:
id2word = corpora.Dictionary(data_words) # Create Dictionary
texts = data_words  # Create Corpus
corpus = [id2word.doc2bow(text) for text in texts] # Term Document Frequency

print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1)]


In [33]:
print(id2word)

Dictionary(8264 unique tokens: ['account', 'bad', 'best', 'better', 'blue']...)


Building base LDA Model

In [45]:
num_topics = 10  # number of topics
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True,
                                       alpha=0.01,
                                       eta=0.9)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.093*"election" + 0.035*"day" + 0.013*"voting" + 0.011*"tomorrow" + '
  '0.008*"night" + 0.008*"time" + 0.006*"early" + 0.006*"watch" + 0.006*"last" '
  '+ 0.006*"fraud"'),
 (1,
  '0.082*"vote" + 0.022*"democrat" + 0.017*"voting" + 0.014*"people" + '
  '0.012*"tomorrow" + 0.012*"get" + 0.012*"red" + 0.011*"republican" + '
  '0.010*"like" + 0.010*"want"'),
 (2,
  '0.179*"vote" + 0.077*"year" + 0.041*"choice" + 0.029*"people" + '
  '0.020*"celebrity" + 0.003*"coming" + 0.002*"music" + 0.002*"fan" + '
  '0.002*"award" + 0.002*"kim"'),
 (3,
  '0.131*"voting" + 0.071*"favorite" + 0.028*"pop" + 0.027*"group" + '
  '0.021*"duo" + 0.018*"year" + 0.011*"harry" + 0.009*"swift" + 0.005*"male" + '
  '0.004*"female"'),
 (4,
  '0.018*"vote" + 0.018*"voting" + 0.015*"democrat" + 0.015*"win" + '
  '0.014*"election" + 0.008*"would" + 0.008*"people" + 0.007*"make" + '
  '0.007*"na" + 0.007*"think"'),
 (5,
  '0.013*"voice" + 0.005*"michigan" + 0.004*"weather" + 0.002*"storm" + '
  '0.002*"obi" +

* Calculating Model perplexity and Coherence score
    * Multiple simulations were conducted with seperate parameters and we identified that at num_topis = 10 , we get the best coherance score for our model.

In [46]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.41753944190016695


Analysis LDA Model Results with pyLDAvis
- Top 10 Topics


In [48]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('visualization/ldavis_prepared_'+str(num_topics))
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, 'visualization/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

c:\Users\pdhiman\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.218347 -0.096637       1        1  44.930584
0     -0.130871  0.112103       2        1  18.734550
9     -0.083850  0.034213       3        1  12.324745
4     -0.098955  0.007331       4        1  11.494336
2      0.079105 -0.132275       5        1   3.916233
3      0.122494  0.023309       6        1   3.765878
7      0.069542  0.024834       7        1   2.073647
8      0.073899 -0.026019       8        1   1.219398
6      0.091156  0.026404       9        1   1.014109
5      0.095827  0.026737      10        1   0.526520, topic_info=           Term         Freq        Total Category  logprob  loglift
18         vote  7376.000000  7376.000000  Default  30.0000  30.0000
20       voting  2673.000000  2673.000000  Default  29.0000  29.0000
125    election  3129.000000  3129.000000  Default  28.0000  28.0000
30     tomorrow  1226.000000  1226.000000  Default  27.0000  27.0000
23     democrat  2640.000000  2640.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1555        per     0.469653    17.795895  Topic10  -7.4589   1.6119
516   education     0.653389    40.366447  Topic10  -7.1287   1.1231
160         oil     0.505670    47.076667  Topic10  -7.3850   0.7130
639    absolute     0.388025    19.913596  Topic10  -7.6498   1.3085
885      likely     0.354571    63.155851  Topic10  -7.7400   0.0642

[533 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1104      1  0.018169  absentee
1104      2  0.926616  absentee
1104      3  0.018169  absentee
1104      4  0.018169  absentee
639       1  0.451953  absolute
...     ...       ...       ...
2051      1  0.065996     youth
2051      2  0.131992     youth
2051      3  0.065996     youth
2051      4  0.263984     youth
2051      7  0.329980     youth

[2120 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 10, 5, 3, 4, 8, 9, 7, 6])